# NUMPY SHAPE SUCKS 

In [3]:

from typing import List, Tuple
from typing_extensions import Self 

In [4]:


LAYERS_MAX = 4
LAYERS_QUAD_NUM = 4
LAYER_SEPARATOR_CHAR = ":"

VOID_SHAPE = "--"
PIN_SHAPE = "P-"
CRYSTAL_SHAPE = "cu"

SLIME_SHAPES = [VOID_SHAPE, PIN_SHAPE]
CRYSTALISED_SHAPES = [VOID_SHAPE, PIN_SHAPE]
CRYSTAL_SHAPES = ['c-', 'cu', 'cr']

SHAPE_NOTHING_CHAR = "-"
SHAPE_FULL_CHAR = "R"
SHAPE_PIN_CHAR = "P"
SHAPE_CRYSTAL_CHAR = "c"



# REF OBJ 

In [5]:
import shapez2_render as Render 
import shapez2_gameinfo as GlobalInfos 
import shapez2_operation as ref


In [6]:
class RefShape:
    def __init__(self, shapeCode:str) -> None:
        self.shape = ref.Shape.fromShapeCode(shapeCode)
  
    def toCode(self) -> str:
        return self.shape.toShapeCode()

    def __repr__(self) -> str:
        
        layers = self.shape.toListOfLayers()
        layers = [    [s[i:i+2] for i in range(0, len(s), 2)] for s in layers]
        h1, h2 = "", ""
        s1 = ""
        s2 = ""
        for l in range(0, len(layers)):
            h1 += "_____    "
            s1 += layers[l][3] + " " + layers[l][0] + "    "
            s2 += layers[l][2] + " " + layers[l][1] + "    "
            h2 += "⎺⎺⎺⎺⎺    "
        return ('\n'.join([h1, s1, s2, h2]))
        # for layer
        # print(layers)
        # layers.reverse()
        # print ("________\n" + '\n'.join(layers) + "\n⎺⎺⎺⎺⎺⎺⎺⎺")

              
def refrot90(s:RefShape) -> RefShape:
    shape2 = ref.rotate90CW(s.shape)[0]
    return RefShape(shape2.toShapeCode())

def refrot270(s:RefShape) -> RefShape:
    shape2 = ref.rotate90CCW(s.shape)[0]
    return RefShape(shape2.toShapeCode())

def refrot180(s:RefShape) -> RefShape:
    shape2 = ref.rotate180(s.shape)[0]
    return RefShape(shape2.toShapeCode())

def refpushpin(s:RefShape) -> RefShape:
    shape2 = ref.pushPin(s.shape)[0]
    return RefShape(shape2.toShapeCode())

def refgenCrystal(s:RefShape) -> RefShape:
    shape2 = ref.genCrystal(s.shape, "u")[0]
    return RefShape(shape2.toShapeCode())


def refcut(s:RefShape) -> list[RefShape]:
    shape = ref.coreCut(s.shape)
    return RefShape(shape[0].toShapeCode()), RefShape(shape[1].toShapeCode())

def refswap(s:RefShape, shapeB:RefShape) -> list[RefShape]:
    shape = ref.swapHalves(s.shape, shapeB.shape)
    return RefShape(shape[0].toShapeCode()), RefShape(shape[1].toShapeCode())

def refstack(s:RefShape, topShape:RefShape) -> RefShape:
    shape = ref.coreStack(s.shape, topShape.shape)
    return RefShape(shape[0].toShapeCode())



# INDEX 

In [7]:
# INDEX

class MemoryIndex:
    
    def __init__(self):
        self.allKey = set()
        self.newKey = []
        self.keyChecked = []
        self.keyToCheck = []
        self.keyOneLayer = []
        self.keyNewOneLayer = []
        self.newLinkList = []

    def initBatch(self, allKey, keyToCheck, keyToStack):
        self.allKey = allKey
        self.keyOneLayer = keyToStack
        self.keyToCheck = keyToCheck

        self.print_db_count()

    # Function to check if a key exists
    def key_exists(self:Self, key: str) -> bool:
        return key in self.allKey

    # Function to get the first key with ischeck = False
    def get_first_keyToCheck(self:Self) -> str:
        return self.keyToCheck.pop() if self.keyToCheck else None

    # Function to set "ischeck" for an existing key
    def set_ischeck(self:Self, key: str) -> None:
        self.keyChecked.append(key)

    # Function to add or update a key/value pair
    def add_key_value(self:Self, key: str, value:object) -> None:
        if key == '--------':
            return
        
        if (key not in self.allKey):
            self.newLinkList.append(value)
            self.newKey.append(value)
            self.allKey.add(key)
            if False:
                if value[1] == 1: #  nlayer
                    self.keyNewOneLayer.append(key)
                if value[1] <= 2: #  nlayer
                    self.keyToCheck.append(key)

    # Function to get all the checked keys
    def print_db_count(self:Self) -> str:
        # Print the counts
        print(f"{'Total Link---':<30}{len(self.newLinkList):<10}")
        print(f"{'Total allKey':<30}{len(self.allKey):<10}")
        print(f"{'Total newKey':<30}{len(self.newKey):<10}")
        print(f"{'Total keyChecked':<30}{len(self.keyChecked):<10}")
        print(f"{'Total keyToCheck':<30}{len(self.keyToCheck):<10}")
        print(f"{'Total keyOneLayer':<30}{len(self.keyOneLayer):<10}")
        return ""


In [8]:
# mIndex = MemoryIndex()
# mIndex.add_key_value('CuCuCuCu', ['CuCuCuCu', 1, 'start', '', 0, '', 0, 0])

# mIndex.print_db_count()


# LOOP TEST REF 

In [9]:
def index_shape(mshape:RefShape, op:str, parentA, rotA:int, parentB: str, rotB:int, opindex:int=0) -> None:
    a = mshape
    b = refrot90(a)
    c = refrot90(b)
    d = refrot90(c)
    
    key = sorted([it.toCode() for it in [a, b , c, d]])[0]
    
    nlayers = mshape.shape.numLayers
    mIndex.add_key_value(key, [key, nlayers, op, parentA, rotA, parentB, rotB, opindex])


In [10]:

def _test_pin(mshape:RefShape):
    shapeCode = mshape.toCode()
    a = refpushpin(mshape)
    index_shape(a, 'pin', parentA=shapeCode, rotA=0, parentB='', rotB=0, opindex=0)   

def _test_genCrytal(mshape:RefShape):
    shapeCode = mshape.toCode()
    a = refgenCrystal(mshape)
    index_shape(a, 'crys', parentA=shapeCode, rotA=0, parentB='', rotB=0, opindex=0)   


def _test_cut(mshape:RefShape):
    shapeCode = mshape.toCode()
    a, b = refcut(mshape)
    index_shape(a, 'cut', parentA=shapeCode, rotA=0, parentB='', rotB=0, opindex=0)   
    index_shape(b, 'cut', parentA=shapeCode, rotA=0, parentB='', rotB=0, opindex=1)   


def _test_swap(mshapeA:RefShape, mshapeB: RefShape):
    shapeCodeA = mshapeA.toCode()
    shapeCodeB = mshapeB.toCode()

    a, b = refswap(mshapeA, mshapeB)
    index_shape(a, 'swap', parentA=shapeCodeA, rotA=0, parentB=shapeCodeB, rotB=0, opindex=0)   
    index_shape(b, 'swap', parentA=shapeCodeA, rotA=0, parentB=shapeCodeB, rotB=0, opindex=1)   


def _test_stak(mshapeA:RefShape, mshapeB: RefShape):
    shapeCodeA = mshapeA.toCode()
    shapeCodeB = mshapeB.toCode()

    a = refstack(mshapeA, mshapeB)
    index_shape(a, 'stack', parentA=shapeCodeA, rotA=0, parentB=shapeCodeB, rotB=0, opindex=0)   


In [11]:


def apply_shape_function():
    shapeCode = mIndex.get_first_keyToCheck()
    if (shapeCode == '--------'): return True
    if (not shapeCode):
        print ('####################### UncheckCode Not Found ##################################### ')
        return False

    mshape = RefShape(shapeCode)
    # index_shape(mshape.rotC(), 'rot', shapeCode, '', 0)   
    
    _test_pin(mshape)
    # index_shape(pushPin(mshape), 'pin', parentA=shapeCode, rotA=0, parentB='', rotB=0, opindex=0)   
    
    _test_genCrytal(mshape)
    
    _test_cut(mshape)
    _test_cut(refrot90(mshape))

    # nlayer = _nlayerShape(mshape.matrix)
    # nlayer = mshape.shape.numLayers
    # if (nlayer > 2): 
    #     mIndex.set_ischeck(shapeCode)
    #     return True

    for it in mIndex.keyOneLayer:
        shapeCodeB = it
        mshapeB = RefShape(shapeCodeB)

        _test_swap(mshape, mshapeB)
        _test_swap(mshape, refrot90(mshapeB))
        _test_swap(refrot90(mshape), refrot90(mshapeB))
        _test_swap(refrot90(mshape), mshapeB)

        _test_stak(mshape, mshapeB)
        _test_stak(mshape, refrot90(mshapeB))
        _test_stak(refrot90(mshape), refrot90(mshapeB))
        _test_stak(refrot90(mshape), mshapeB)

        _test_stak(mshapeB, mshape)
        _test_stak(mshapeB, refrot90(mshape))
        _test_stak(refrot90(mshapeB), refrot90(mshape))
        _test_stak(refrot90(mshapeB), mshape)


    mIndex.set_ischeck(shapeCode)
    return True


In [12]:
from IPython.display import clear_output
import time

if False : # Frist Loop 
    i = 0
    noFinish = True
    while noFinish:
        i += 1
        isapply =apply_shape_function()
        if (not isapply):
            noFinish = False
        if (i % 100 == 0):
            # clear_output(wait=True)
            print('---------------------------', i)
            mIndex.print_db_count()
    mIndex.print_db_count()
    # print_db(10)



In [13]:
import sqlite3

# name = 'key_2_on_1_A'

class MDataBase:

    def __init__(self, name:str):
        # Connect to SQLite database (or create it if it doesn't exist)
        self.conn = sqlite3.connect('data/' + name + '.db')

        # Create a cursor object
        self.cur = self.conn.cursor()

        # Create a table to store key/value pairs if it doesn't exist
        self.cur.execute('''
            CREATE TABLE IF NOT EXISTS key_value_store (
                key TEXT PRIMARY KEY,
                nlayer INT,

                operator TEXT,
                opindex INTEGER,

                parentA TEXT,
                rotationA INT,
                
                parentB TEXT, 
                rotationB INT,
                ischeck BOOLEAN
            )
        ''')

    def commit(self):
        self.conn.commit()  

    def add_key_value(self, data:list) -> None:
        self.cur.execute('''
            INSERT OR REPLACE INTO key_value_store (key, nlayer, operator, parentA, rotationA, parentB, rotationB, opindex, ischeck)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, false)
        ''', (data[0],  data[1], data[2], data[3], data[4], data[5], data[6], data[7]))
    
    def get_all_keys(self) -> None:
        self.cur.execute('''
            SELECT DISTINCT(key) FROM key_value_store
        ''', ())
        rows = self.cur.fetchall()
        return [row[0] for row in rows]


    def get_n_layer_shapes(self, n:int) -> None:
        self.cur.execute('''
            SELECT DISTINCT(key) FROM key_value_store
            WHERE nlayer = ?
        ''', (n, ))
        rows = self.cur.fetchall()
        return [row[0] for row in rows]
                    

    def search_key(self, key:str) -> None:
        self.cur.execute('''
            SELECT * FROM key_value_store
            WHERE key = ?
        ''', (key, ))
        row = self.cur.fetchone()
        print(row)
        # key, nlayer, operator, parentA, rotationA, parentB, rotationB, opindex, ischeck = row
        return row

    def printLayer(self) -> None:
        print ("1 layers : ", len(self.get_n_layer_shapes(1)))
        print ("2 layers : ", len(self.get_n_layer_shapes(2)))
        print ("3 layers : ", len(self.get_n_layer_shapes(3)))
        print ("4 layers : ", len(self.get_n_layer_shapes(4)))


In [24]:
# tdb = MDataBase('key_3(2)_on_1_#')
# tdb = MDataBase('key_3(2)_on_1_#')
# tdb = MDataBase('key_3(2)_on_1_#')
# tdb = MDataBase('key_3(2)_on_1_#')
# tdb = MDataBase('key_3(2)_on_1_#')
# tdb = MDataBase('key_3(2)_on_1_#')
# tdb = MDataBase('key_1_on_1')
mdbs = [
    MDataBase('key_1_on_1'),
    # MDataBase('key_2_on_1_A'),
    MDataBase('key_2_on_1_B'),
    MDataBase('key_2_on_1_C'),
    MDataBase('key_2_on_1_D'),
    MDataBase('key_2_on_1_E'),
    MDataBase('key_3_on_1_#'),
    MDataBase('key_3(2)_on_1_#'),
    MDataBase('key_2_on_1_A'),
]


# tdb.printLayer()

# tdb.get_n_layer_shapes(3)
tdb.search_key('P-------:CcCcCcCc:----cw--')





None


TypeError: cannot unpack non-iterable NoneType object

In [ ]:
([
    'CuP-cuP-:--cu----',
    '------cu', 
    '--cuP-Cu:--cuCuCu:Cu--CuCu',
])


In [123]:
('--cuCucu:------cu', 2, 'swap', 0, '--cuP-Cu:--cuCuCu:Cu--CuCu', 0, '------cu', 0, 0)
('--cuP-Cu:--cuCuCu:Cu--CuCu', 3, 'stack', 0, '--cuP-Cu:--cuCuCu', 0, 'Cu--CuCu', 0, 0)
('--cuP-Cu:--cuCuCu', 2, 'stack', 0, '--cuP-Cu:--cu----', 0, '----CuCu', 0, 0)
('--cuP-Cu:--cu----', 2, 'swap', 1, '--CucuP-:--Cucu--', 0, 'Cu----Cu', 0, 0)
('--CucuP-:--Cucu--', 2, 'swap', 1, 'CucuCucu:CuCuCucu', 0, 'P---P-cu', 0, 0)
('CucuCucu:CuCuCucu', 2, 'crys', 0, 'CucuCucu:Cu--CuCu', 0, '', 0, 0)
('CucuCucu:Cu--CuCu', 2, 'stack', 0, 'cuCucuCu', 0, 'cuCuCuCu', 0, 0)


('--cuCucu:------cu',
 2,
 'swap',
 0,
 '--cuP-Cu:--cuCuCu:Cu--CuCu',
 0,
 '------cu',
 0,
 0)

1 layers :  63
2 layers :  3358
3 layers :  84380
4 layers :  0


# LOOP TO TEST 

In [22]:
mdbs = [
    MDataBase('key_1_on_1'),
    # MDataBase('key_2_on_1_A'),
    MDataBase('key_2_on_1_B'),
    MDataBase('key_2_on_1_C'),
    MDataBase('key_2_on_1_D'),
    MDataBase('key_2_on_1_E'),
    MDataBase('key_3_on_1_#'),
    MDataBase('key_3(2)_on_1_#'),
    MDataBase('key_2_on_1_A'),
]


all_keys = set([k  for it in mdbs for k in it.get_all_keys()])
stack_keys_list = [k  for it in mdbs for k in it.get_n_layer_shapes(1)]
key_to_check = mdbs[len(mdbs) - 1].get_n_layer_shapes(3)

print ('all_keys       ', len(all_keys))
print ('stack_keys_list', len(stack_keys_list))
print ('key_to_check   ', len(key_to_check))

mIndex = MemoryIndex()
mIndex.initBatch(all_keys, key_to_check, stack_keys_list)

all_keys        2632970
stack_keys_list 63
key_to_check    25726
Total Link---                 0         
Total allKey                  2632970   
Total newKey                  0         
Total keyChecked              0         
Total keyToCheck              25726     
Total keyOneLayer             63        


In [23]:

print('-start-')

i = 0
noFinish = True
while noFinish:
    i += 1
    isapply =apply_shape_function()
    if (not isapply):
        noFinish = False
    if (i % 100 == 0):
        # clear_output(wait=True)
        print('---------------------------', i)
        mIndex.print_db_count()
mIndex.print_db_count()
# print_db(10)




-start-
--------------------------- 100
Total Link---                 2500      
Total allKey                  2635470   
Total newKey                  2500      
Total keyChecked              100       
Total keyToCheck              25626     
Total keyOneLayer             63        
--------------------------- 200
Total Link---                 4506      
Total allKey                  2637476   
Total newKey                  4506      
Total keyChecked              200       
Total keyToCheck              25526     
Total keyOneLayer             63        
--------------------------- 300
Total Link---                 6198      
Total allKey                  2639168   
Total newKey                  6198      
Total keyChecked              300       
Total keyToCheck              25426     
Total keyOneLayer             63        
--------------------------- 400
Total Link---                 7151      
Total allKey                  2640121   
Total newKey                  7151      
Tot

''

In [17]:
mIndex.print_db_count()

Total Link---                 14        
Total allKey                  2632970   
Total newKey                  14        
Total keyChecked              9         
Total keyToCheck              0         
Total keyOneLayer             63        


''

In [18]:
newdb=  MDataBase('key_3(2)_on_1_#')



In [19]:
for it in mIndex.newLinkList:
    newdb.add_key_value(it)
newdb.commit()    

In [ ]:


class TrieNode:
    def __init__(self):
        self.children = {}
        self.is_end_of_word = False

class Trie:
    def __init__(self):
        self.root = TrieNode()

    def insert(self, key):
        node = self.root
        for char in key:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
        node.is_end_of_word = True

    def search(self, key):
        node = self.root
        for char in key:
            if char not in node.children:
                return False
            node = node.children[char]
        return node.is_end_of_word

# Usage
trie = Trie()
trie.insert("key1")
trie.insert("key2")

print(trie.search("key1"))  # True
print(trie.search("key3"))  # False